In [ ]:
!pip3 install scikit-image --user

In [1]:
import os
import keras
import numpy as np
import random
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten
from keras.models import model_from_json

from skimage import transform
from skimage.io import imread
from skimage.color import rgb2gray
from sklearn.metrics import classification_report, confusion_matrix

Using TensorFlow backend.


Couldn't import dot_parser, loading of dot files will not be possible.


In [2]:
"""
    Preprocess image, normalizing and resizing it

    :param frame: RGBA frame
"""    
def preprocess_image(frame):
    
    # Normalize Pixel Values
    normalized_frame = frame/255.0 - 0.5
    
    # Resize
    preprocessed_frame = transform.resize(normalized_frame, IMAGE_PP_SIZE)
    
    return preprocessed_frame

"""
    Create 2D label list from 1D list
    
    :param labels: 1D label list
"""

def make_labels(labels):
    np_labels = np.zeros((len(labels), 16))
    
    for i in range(len(labels)):
        np_labels[i, labels[i]] = 1
    
    return np_labels

def file_len(fname):
    with open(fname) as f:
        for i, l in enumerate(f):
            pass
    return i + 1

def choices(l, k=1):
    new_list = []
    for i in range(k):
        new_list.append(random.choice(l))
    return new_list

def build_model():
    model = Sequential()
    model.add(Conv2D(32, (3,3),
                     activation='relu',
                     input_shape=(IMAGE_PP_SIZE[0], IMAGE_PP_SIZE[1], 1)))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(128, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(128, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Flatten())
    model.add(Dense(1024, activation='relu'))
    model.add(Dense(1024, activation='relu'))
    model.add(Dense(16, activation='softmax'))
    
    return model

## Define Globals

In [12]:
IMAGE_PP_SIZE = [150, 150]
EPOCHS = 10
BATCH_SIZE = 100
TRAIN_STEP = 1000
VAL_SIZE = 100

RELOAD_MODEL = False

## Load Model if it Exists, Otherwise Recreate it

In [4]:
if RELOAD_MODEL:
    try:
        json_file = open('model_complex.json', 'r')
        loaded_model_json = json_file.read()
        json_file.close()
        model = model_from_json(loaded_model_json)
        # load weights into new model
        model.load_weights("model_complex.h5")
    except FileNotFoundError:
        model = build_model()
else:
    model = build_model()

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.SGD(lr=0.01),
              metrics=['accuracy'])

In [16]:

#
# Define train, validation and test lists
#

train_labels = []
val_labels = []
test_labels = []

train_input = []
val_input = []
test_input = []

#
# Load validation dataset
#

folder = "/media/matheusmortatti/External/rvl-cdip"
labels_file = open(folder + "/labels/val.txt")

val_file = []
for line in labels_file:
    sp = line.split()
    val_file.append(sp)
    
#
# Open training list file and start training
#


trained = 0
train_size = file_len(folder + "/labels/train.txt")

for i in range(EPOCHS):
    trained = 0
    labels_file = open(folder + "/labels/train.txt")
    for line in labels_file:
        sp = line.split()
        pp_img = preprocess_image(imread(folder + "/images/" + sp[0]))

        train_input.append(pp_img)
        train_labels.append(int(sp[1]))

        if len(train_input) >= TRAIN_STEP:
            
            trained += len(train_input)

            #
            # Choose a subset of the validation data
            #

            ss_val = choices(val_file, k=VAL_SIZE)

            for v in ss_val:
                val_input.append(preprocess_image(imread(folder + "/images/" + v[0])))
                val_labels.append(int(v[1]))

            x_val = np.expand_dims(np.asarray(val_input), axis=3)
            y_val = make_labels(val_labels)

            #
            # Create training data
            #

            x_train = np.expand_dims(np.asarray(train_input), axis=3)
            y_train = make_labels(train_labels)

            model.fit(x_train, y_train,
                      batch_size=BATCH_SIZE,
                      epochs=1,
                      verbose=1,
                      validation_data=(x_val, y_val))

            #
            # Reset lists for next iteration
            #

            train_input = []
            train_labels = []
            val_input = []
            val_labels = []
            
            print("trained: " + str(trained) + " / " + str(train_size))
    labels_file.close()


/home/matheusmortatti/.local/lib/python3.5/site-packages/skimage/transform/_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/home/matheusmortatti/.local/lib/python3.5/site-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "


Train on 1000 samples, validate on 100 samples
Epoch 1/1
1000/1000 [==============================] - 1s 1ms/step - loss: 0.6594 - acc: 0.8060 - val_loss: 0.6052 - val_acc: 0.7900
trained: 1000 / 320000
Train on 1000 samples, validate on 100 samples
Epoch 1/1
1000/1000 [==============================] - 1s 1ms/step - loss: 0.6713 - acc: 0.7960 - val_loss: 0.8252 - val_acc: 0.7300
trained: 2000 / 320000
Train on 1000 samples, validate on 100 samples
Epoch 1/1
1000/1000 [==============================] - 1s 1ms/step - loss: 0.6383 - acc: 0.8170 - val_loss: 0.6209 - val_acc: 0.7800
trained: 3000 / 320000
Train on 1000 samples, validate on 100 samples
Epoch 1/1
1000/1000 [==============================] - 1s 1ms/step - loss: 0.6822 - acc: 0.7970 - val_loss: 0.6286 - val_acc: 0.8100
trained: 4000 / 320000
Train on 1000 samples, validate on 100 samples
Epoch 1/1
1000/1000 [==============================] - 1s 1ms/step - loss: 0.6508 - acc: 0.8010 - val_loss: 0.7418 - val_acc: 0.7600
trained:

1000/1000 [==============================] - 1s 1ms/step - loss: 0.5925 - acc: 0.8280 - val_loss: 0.6945 - val_acc: 0.7900
trained: 41000 / 320000
Train on 1000 samples, validate on 100 samples
Epoch 1/1
1000/1000 [==============================] - 1s 1ms/step - loss: 0.6521 - acc: 0.8070 - val_loss: 0.6061 - val_acc: 0.7800
trained: 42000 / 320000
Train on 1000 samples, validate on 100 samples
Epoch 1/1
1000/1000 [==============================] - 1s 1ms/step - loss: 0.6699 - acc: 0.8100 - val_loss: 0.6760 - val_acc: 0.8000
trained: 43000 / 320000
Train on 1000 samples, validate on 100 samples
Epoch 1/1
1000/1000 [==============================] - 1s 1ms/step - loss: 0.6224 - acc: 0.8110 - val_loss: 0.4628 - val_acc: 0.8500
trained: 44000 / 320000
Train on 1000 samples, validate on 100 samples
Epoch 1/1
1000/1000 [==============================] - 1s 1ms/step - loss: 0.7047 - acc: 0.7810 - val_loss: 0.6890 - val_acc: 0.7800
trained: 45000 / 320000
Train on 1000 samples, validate on 10

1000/1000 [==============================] - 1s 1ms/step - loss: 0.7054 - acc: 0.7940 - val_loss: 0.7881 - val_acc: 0.7300
trained: 81000 / 320000
Train on 1000 samples, validate on 100 samples
Epoch 1/1
1000/1000 [==============================] - 1s 1ms/step - loss: 0.6016 - acc: 0.8180 - val_loss: 0.7108 - val_acc: 0.7900
trained: 82000 / 320000
Train on 1000 samples, validate on 100 samples
Epoch 1/1
1000/1000 [==============================] - 1s 1ms/step - loss: 0.5890 - acc: 0.8180 - val_loss: 0.7585 - val_acc: 0.7800
trained: 83000 / 320000
Train on 1000 samples, validate on 100 samples
Epoch 1/1
1000/1000 [==============================] - 1s 1ms/step - loss: 0.6546 - acc: 0.8110 - val_loss: 0.6904 - val_acc: 0.8300
trained: 84000 / 320000
Train on 1000 samples, validate on 100 samples
Epoch 1/1
1000/1000 [==============================] - 1s 1ms/step - loss: 0.7016 - acc: 0.7880 - val_loss: 0.9423 - val_acc: 0.7700
trained: 85000 / 320000
Train on 1000 samples, validate on 10

KeyboardInterrupt: 

## Save Model and Weights

In [15]:
model_json = model.to_json()
with open("model_complex.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model_complex.h5")

In [ ]:
np_img_type = np.zeros((len(img_type), dt))

print(len(img_type))

for i in range(len(img_type)):
    np_img_type[i, img_type[i]] = 1
print(np_img_type)

In [ ]:
x_train = np.expand_dims(np.asarray(imgs[:2000]), axis=3)
y_train = np_img_type[:2000,:]

x_val = np.expand_dims(np.asarray(imgs[2000:4000]), axis=3)
y_val = np_img_type[2000:4000,:]

x_test = np.expand_dims(np.asarray(imgs[4000:]), axis=3)
y_test = np_img_type[4000:,:]

print(y_train.shape)

In [ ]:
model.fit(x_train, y_train,
          batch_size=100,
          epochs=2,
          verbose=1,
          validation_data=(x_val, y_val))

In [ ]:
#Confution Matrix and Classification Report
Y_pred = model.predict_classes(x_test)

In [ ]:
print(Y_pred)

In [ ]:
y_pred = Y_pred
y_true = np.argmax(y_test, axis=1)
print('Confusion Matrix')
print(confusion_matrix(y_true, y_pred))